In [5]:
import torch
print("CUDA disponible:", torch.cuda.is_available())
print("Versión de CUDA:", torch.version.cuda)
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No detectada")
print("CUDA disponible:", torch.cuda.is_available())
print("Número de GPUs:", torch.cuda.device_count())
torch.backends.cudnn.benchmark = True  # Optimiza el uso de la GPU
torch.cuda.empty_cache()  # Libera memoria de GPU antes de entrenar
print("Versión de PyTorch:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
print("Versión de CUDA en PyTorch:", torch.version.cuda)
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No detectada")

CUDA disponible: False
Versión de CUDA: None
Nombre de la GPU: No detectada
CUDA disponible: False
Número de GPUs: 0
Versión de PyTorch: 2.6.0+cpu
CUDA disponible: False
Versión de CUDA en PyTorch: None
Nombre de la GPU: No detectada


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Transformaciones para normalizar imágenes (Requeridas por ResNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar imágenes
    transforms.ToTensor(),  # Convertir a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización de ResNet
])
# Rutas a los conjuntos de entrenamiento y validación
train_dir = "./Data/train"
valid_dir = "./Data/valid"
# Cargar imágenes desde carpetas (PyTorch usa estructura basada en subcarpetas para cada clase)
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)
# Definir DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
# Verificar categorías detectadas
print("Clases detectadas:", train_dataset.classes)
# Definir modelo y modificar la última capa
num_classes = 6  # Número de categorías
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # Ajustar capa final
model = model.to(device)
# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Entrenamiento del modelo
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Acc: {acc:.2f}%, Time: {time.time() - start_time:.2f}s")
# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Clases detectadas: ['freshripe', 'freshunripe', 'overripe', 'ripe', 'rotten', 'unripe']
Epoch 1/10, Loss: 0.3148, Acc: 89.09%, Time: 25.41s
Epoch 2/10, Loss: 0.1638, Acc: 95.06%, Time: 25.38s
Epoch 3/10, Loss: 0.1018, Acc: 96.73%, Time: 25.56s
Epoch 4/10, Loss: 0.1105, Acc: 96.31%, Time: 25.25s
Epoch 5/10, Loss: 0.0890, Acc: 97.40%, Time: 25.42s
Epoch 6/10, Loss: 0.0910, Acc: 96.71%, Time: 25.65s


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000204A6A140D0>
Traceback (most recent call last):
  File "c:\Users\Raxie\anaconda3\envs\IA\lib\site-packages\torch\utils\data\dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "c:\Users\Raxie\anaconda3\envs\IA\lib\site-packages\torch\utils\data\dataloader.py", line 1562, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
